#Preparação

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

In [3]:
!tar xf /content/spark-3.5.3-bin-hadoop3.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

In [5]:
!pip install -q findspark

In [124]:
import findspark
findspark.init()

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkcolab')\
        .getOrCreate()

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [26]:
logs = spark.read.csv('/content/E-commerce Website Logs.csv', inferSchema=True, header=True)
logs.show(truncate=False)

+-----------------------+---------------+----------------+-------------+------+---------------+---+------+-------+----------+--------+--------+--------+---------------+-----------+
|accessed_date          |duration_(secs)|network_protocol|ip           |bytes |accessed_Ffom  |age|gender|country|membership|language|sales   |returned|returned_amount|pay_method |
+-----------------------+---------------+----------------+-------------+------+---------------+---+------+-------+----------+--------+--------+--------+---------------+-----------+
|2017-03-14 17:43:57.172|2533           |TCP             |1.10.195.126 |20100 |Chrome         |28 |Female|CA     |Normal    |English |261.96  |No      |0.0            |Credit Card|
|2017-03-14 17:43:57.172|4034           |TCP             |1.1.217.211  |20500 |Mozilla Firefox|21 |Male  |AR     |Normal    |English |731.94  |No      |0.0            |Debit Card |
|2017-03-14 17:43:26.135|1525           |TCP             |1.115.198.107|90100 |Mozilla Firefox|

#Tratamento

In [27]:
logs = logs.withColumn("hora", hour(col("accessed_date"))) \
       .withColumn("dia_semana", dayofweek(col("accessed_date"))) \
       .withColumn("mes", month(col("accessed_date")))

In [29]:
logs.show(2)

+--------------------+---------------+----------------+------------+-----+---------------+---+------+-------+----------+--------+------+--------+---------------+-----------+----+----------+---+
|       accessed_date|duration_(secs)|network_protocol|          ip|bytes|  accessed_Ffom|age|gender|country|membership|language| sales|returned|returned_amount| pay_method|hora|dia_semana|mes|
+--------------------+---------------+----------------+------------+-----+---------------+---+------+-------+----------+--------+------+--------+---------------+-----------+----+----------+---+
|2017-03-14 17:43:...|           2533|           TCP  |1.10.195.126|20100|         Chrome| 28|Female|     CA|    Normal| English|261.96|      No|            0.0|Credit Card|  17|         3|  3|
|2017-03-14 17:43:...|           4034|           TCP  | 1.1.217.211|20500|Mozilla Firefox| 21|  Male|     AR|    Normal| English|731.94|      No|            0.0| Debit Card|  17|         3|  3|
+--------------------+--------

In [30]:
logs = logs.withColumnsRenamed({'accessed_date':'data_acesso','duration_(secs)':'duracao','accessed_Ffom':'navegador','age':'idade','gender':'genero',\
                        'country':'pais','membership':'nivel','language':'idioma','sales':'vendas','pay_method':'metodo_pagamento'})

In [31]:
logs = logs.select('data_acesso','duracao','navegador','idade','genero','pais','nivel','idioma','vendas','metodo_pagamento','hora','dia_semana','mes')
logs.show(5,truncate=False)

+-----------------------+-------+---------------+-----+------+----+------+-------+--------+----------------+----+----------+---+
|data_acesso            |duracao|navegador      |idade|genero|pais|nivel |idioma |vendas  |metodo_pagamento|hora|dia_semana|mes|
+-----------------------+-------+---------------+-----+------+----+------+-------+--------+----------------+----+----------+---+
|2017-03-14 17:43:57.172|2533   |Chrome         |28   |Female|CA  |Normal|English|261.96  |Credit Card     |17  |3         |3  |
|2017-03-14 17:43:57.172|4034   |Mozilla Firefox|21   |Male  |AR  |Normal|English|731.94  |Debit Card      |17  |3         |3  |
|2017-03-14 17:43:26.135|1525   |Mozilla Firefox|20   |Male  |PL  |Normal|English|14.62   |Cash            |17  |3         |3  |
|2017-03-14 17:43:26.135|4572   |Mozilla Firefox|66   |Female|IN  |Normal|Spanish|957.5775|Credit Card     |17  |3         |3  |
|2017-03-14 18:17:09.005|3652   |Mozilla Firefox|53   |Female|KR  |Normal|Spanish|22.368  |Cash  

In [47]:
logs.printSchema()

root
 |-- data_acesso: timestamp (nullable = true)
 |-- duracao: integer (nullable = true)
 |-- navegador: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- nivel: string (nullable = true)
 |-- idioma: string (nullable = true)
 |-- vendas: double (nullable = true)
 |-- metodo_pagamento: string (nullable = true)
 |-- hora: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- mes: integer (nullable = true)



In [46]:
logs = logs.withColumn('idade',col('idade').cast(IntegerType()))

In [71]:
for coluna in logs.columns:
  print(coluna, logs.filter(logs[coluna].isNull()).count())

data_acesso 0
duracao 0
navegador 0
idade 0
genero 0
pais 0
nivel 0
idioma 0
vendas 0
metodo_pagamento 0
hora 0
dia_semana 0
mes 0


In [57]:
logs.filter(col('nivel') == "Not Logged In").count()

15886

In [68]:
logs = logs.filter(col('idade').isNotNull())


In [70]:
logs.groupBy('genero').count().show()

+------+-----+
|genero|count|
+------+-----+
|Female|50682|
|  Male|34032|
+------+-----+



In [76]:
logs.show(1,truncate=False)

+-----------------------+-------+---------+-----+------+----+------+-------+------+----------------+----+----------+---+
|data_acesso            |duracao|navegador|idade|genero|pais|nivel |idioma |vendas|metodo_pagamento|hora|dia_semana|mes|
+-----------------------+-------+---------+-----+------+----+------+-------+------+----------------+----+----------+---+
|2017-03-14 17:43:57.172|2533   |Chrome   |28   |Female|CA  |Normal|English|261.96|Credit Card     |17  |3         |3  |
+-----------------------+-------+---------+-----+------+----+------+-------+------+----------------+----+----------+---+
only showing top 1 row



#Perguntas do projeto

#Pergunta 1
Quais são os 5 maiores horários de pico de acesso à plataforma durante a semana?

In [112]:
top5_pico = logs.groupBy(col('hora')).agg(count(col('dia_semana'))).orderBy(desc('count(dia_semana)'))
top5_pico.withColumnRenamed('count(dia_semana)','acessos').show(5)

+----+-------+
|hora|acessos|
+----+-------+
|   2|   5880|
|   1|   5656|
|   3|   4968|
|   4|   4767|
|   0|   4341|
+----+-------+
only showing top 5 rows



#Pergunta 2
Existe uma diferença significativa na duração média das sessões entre usuários de diferentes navegadores?

In [127]:
duracao_por_navegador = logs.groupBy('navegador').agg(mean('duracao')).orderBy(desc('avg(duracao)')).show()

+---------------+------------------+
|      navegador|      avg(duracao)|
+---------------+------------------+
| Microsoft Edge|3262.5964505119455|
|    Android App| 3258.735507246377|
|         Chrome| 3256.836589955131|
|        IOS App| 3255.549261315517|
|         Safari|3246.3750309329375|
|         Others| 3245.094103175213|
|Mozilla Firefox|3243.1286976565502|
|         SafFRi| 3236.527777777778|
+---------------+------------------+



#Pergunta 3

Qual o método de pagamento mais utilizado pelos usuários em cada país?

In [138]:
pais = logs.select('pais','metodo_pagamento')
pais.show(5)

+----+----------------+
|pais|metodo_pagamento|
+----+----------------+
|  CA|     Credit Card|
|  AR|      Debit Card|
|  PL|            Cash|
|  IN|     Credit Card|
|  KR|            Cash|
+----+----------------+
only showing top 5 rows



In [161]:
pais_metodo = pais.groupBy('pais','metodo_pagamento').count().orderBy(desc('count'))
pais_metodo.show(20)

+----+----------------+-----+
|pais|metodo_pagamento|count|
+----+----------------+-----+
|  IT|            Cash| 7177|
|  US|            Cash| 5978|
|  IT|     Credit Card| 5132|
|  US|     Credit Card| 4580|
|  CA|            Cash| 3629|
|  CA|     Credit Card| 2670|
|  IT|      Debit Card| 2657|
|  US|      Debit Card| 2372|
|  JP|            Cash| 2105|
|  CN|            Cash| 2091|
|  RU|            Cash| 1973|
|  IT|          Others| 1926|
|  CH|            Cash| 1702|
|  US|          Others| 1681|
|  IN|            Cash| 1660|
|  CN|     Credit Card| 1553|
|  JP|     Credit Card| 1466|
|  RU|     Credit Card| 1424|
|  CA|      Debit Card| 1419|
|  PL|            Cash| 1347|
+----+----------------+-----+
only showing top 20 rows



In [162]:
rank_pagamento_país = Window.partitionBy('pais').orderBy(desc('count'))
pais_metodo = pais_metodo.withColumn('Rank', row_number().over(rank_pagamento_país))
pais_metodo.show()

+----+----------------+-----+----+
|pais|metodo_pagamento|count|Rank|
+----+----------------+-----+----+
|  AE|            Cash|   51|   1|
|  AE|     Credit Card|   32|   2|
|  AE|          Others|   14|   3|
|  AE|      Debit Card|   11|   4|
|  AR|            Cash|  658|   1|
|  AR|     Credit Card|  480|   2|
|  AR|      Debit Card|  256|   3|
|  AR|          Others|  187|   4|
|  AT|            Cash|  948|   1|
|  AT|     Credit Card|  751|   2|
|  AT|      Debit Card|  388|   3|
|  AT|          Others|  255|   4|
|  AU|            Cash|  979|   1|
|  AU|     Credit Card|  663|   2|
|  AU|      Debit Card|  345|   3|
|  AU|          Others|  240|   4|
|  CA|            Cash| 3629|   1|
|  CA|     Credit Card| 2670|   2|
|  CA|      Debit Card| 1419|   3|
|  CA|          Others|  977|   4|
+----+----------------+-----+----+
only showing top 20 rows



In [160]:
pais_metodo=pais_metodo.filter(col('Rank')==1)
pais_metodo = pais_metodo.select('pais','metodo_pagamento','count').orderBy(desc('count'))
pais_metodo.show(50)

+----+----------------+-----+
|pais|metodo_pagamento|count|
+----+----------------+-----+
|  IT|            Cash| 7177|
|  US|            Cash| 5978|
|  CA|            Cash| 3629|
|  JP|            Cash| 2105|
|  CN|            Cash| 2091|
|  RU|            Cash| 1973|
|  CH|            Cash| 1702|
|  IN|            Cash| 1660|
|  PL|            Cash| 1347|
|  GB|            Cash|  985|
|  AU|            Cash|  979|
|  AT|            Cash|  948|
|  SE|            Cash|  925|
|  NO|            Cash|  843|
|  AR|            Cash|  658|
|  KR|            Cash|  471|
|  DE|            Cash|  449|
|  PR|            Cash|  360|
|  CO|            Cash|  296|
|  ZA|            Cash|  263|
|  MX|            Cash|  235|
|  PE|            Cash|  235|
|  FI|            Cash|  145|
|  IE|            Cash|   55|
|  AE|            Cash|   51|
|  DK|     Credit Card|   40|
|  FR|            Cash|   27|
+----+----------------+-----+



#Pegunta 4
Qual país compra mais?

In [175]:
pais_compra = logs.groupBy('pais').agg(sum('vendas')).orderBy(desc('sum(vendas)'))
pais_compra = pais_compra.withColumn('vendas', round(col('sum(vendas)')))
pais_compra = pais_compra.select('pais','vendas')
pais_compra.show()

+----+---------+
|pais|   vendas|
+----+---------+
|  IT|7127469.0|
|  US|6300334.0|
|  CA|3605861.0|
|  CN|2108403.0|
|  JP|2098233.0|
|  RU|1898916.0|
|  IN|1685979.0|
|  CH|1671289.0|
|  PL|1430061.0|
|  GB|1026268.0|
|  AT| 996015.0|
|  SE| 940002.0|
|  AU| 918088.0|
|  NO| 771479.0|
|  AR| 629667.0|
|  DE| 460482.0|
|  KR| 458614.0|
|  PR| 354292.0|
|  CO| 306324.0|
|  ZA| 260920.0|
+----+---------+
only showing top 20 rows

